In [1]:
from __future__ import print_function

import os
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt

import tensorflow.keras
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

from sklearn import preprocessing

### support function

In [2]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"

In [3]:
def preprocess(image, imageWidth, imageHeight) :
    return cv2.resize(image, (imageWidth, imageHeight))

In [4]:
def loadData(path, augment = False, preprocess = None) :
    x = None
    y = np.empty(shape=(0,))
    subDirs = [ name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name)) ]
    
    for classNumber, subDir in enumerate(subDirs):
        tempX = []
        tempY = []
        classDirPath = os.path.join(path, subDir)
        imageDir = [ name for name in os.listdir(classDirPath) if os.path.isfile(os.path.join(classDirPath, name)) ]
        for index, imageName in  enumerate(imageDir) :
            image = cv2.imread(os.path.join(classDirPath, imageName))
            if (preprocess) :
                image = preprocess[0](image, preprocess[1], preprocess[2])
            
            tempX.insert(index,image)
            tempY.insert(index,subDir)
#             break;
        if x is None:
            x = np.array(tempX)
        else :
            x = np.append(x, np.array(tempX), axis = 0)
        y = np.append(y, np.array(tempY), axis = 0)
    return x, y

### model function

In [5]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [6]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

### resnet_v1

In [7]:
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = tensorflow.keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

### resnet_v2

In [8]:
def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = tensorflow.keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

### main

#### load data

In [13]:
trainPath = os.path.join("./seti-data/primary_small/train")
valPath = os.path.join("./seti-data/primary_small/valid")
testPath = os.path.join("./seti-data/primary_small/test")

IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128

(xTrain,yTrainStr) = loadData(trainPath, preprocess = [preprocess, IMAGE_WIDTH,  IMAGE_HEIGHT])
(xVal, yValStr) = loadData(valPath, preprocess = [preprocess, IMAGE_WIDTH,  IMAGE_HEIGHT])
(xTest, yTestStr) = loadData(testPath, preprocess = [preprocess, IMAGE_WIDTH,  IMAGE_HEIGHT])

In [14]:
le = preprocessing.LabelEncoder()
yTrain = le.fit_transform(yTrainStr)
yVal = le.transform(yValStr)
yTest = le.transform(yTestStr)

In [15]:
# predifine
NUM_CLASSES = np.unique(yTrain).shape[0] # 7

# Subtracting pixel mean improves accuracy
SUBTRACT_PIXEL_MEAN = True


COLORS = xTrain.shape[3] # 3

In [16]:
# Input image dimensions.
input_shape = xTrain.shape[1:]

# Normalize data.
xTrain = xTrain.astype('float32') / 255
xVal = xVal.astype('float32') / 255
xTest = xTest.astype('float32') / 255

# If subtract pixel mean is enabled
if SUBTRACT_PIXEL_MEAN:
    xTrainNMean = np.mean(xTrain, axis=0)
    xTrain -= xTrainNMean
    xVal -= xTrainNMean
    xTest -= xTrainNMean
    
print('xTrainN shape:', xTrain.shape)
print(xTrain.shape[0], 'train samples')
print(xVal.shape[0], 'validation samples')
print(xTest.shape[0], 'test samples')
print('yTrain shape:', yTrain.shape)

# Convert class vectors to binary class matrices.
yTrain = tensorflow.keras.utils.to_categorical(yTrain, NUM_CLASSES)
yVal = tensorflow.keras.utils.to_categorical(yVal, NUM_CLASSES)
yTest = tensorflow.keras.utils.to_categorical(yTest, NUM_CLASSES)

xTrainN shape: (5600, 128, 128, 3)
5600 train samples
700 validation samples
700 test samples
yTrain shape: (5600,)


In [21]:
import pickle

pickleOut = open("./seti-data/128xTrain.pickle", "wb")
pickle.dump(xTrain, pickleOut)
pickleOut.close()

pickleOut = open("./seti-data/128yTrain.pickle", "wb")
pickle.dump(yTrain, pickleOut)
pickleOut.close()

pickleOut = open("./seti-data/128xVal.pickle", "wb")
pickle.dump(xVal, pickleOut)
pickleOut.close()

pickleOut = open("./seti-data/128yVal.pickle", "wb")
pickle.dump(yVal, pickleOut)
pickleOut.close()

pickleOut = open("./seti-data/128xTest.pickle", "wb")
pickle.dump(xTest, pickleOut)
pickleOut.close()

pickleOut = open("./seti-data/128yTest.pickle", "wb")
pickle.dump(yTest, pickleOut)
pickleOut.close()

### reshnet1

In [ ]:
BATCH_SIZE = 32 #32  # orig paper trained all networks with batch_size=128
EPOCHS = 200 # 200
USE_AUGMENTATION = True

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
VERSION = 1

# Computed depth from supplied model parameter n
if VERSION == 1:
    DEPTH = COLORS * 6 + 2
elif version == 2:
    DEPTH = COLORS * 9 + 2

#### training param

In [ ]:
# Create the neural network
if VERSION == 2:
    model = resnet_v2(input_shape=input_shape, depth=DEPTH, num_classes=NUM_CLASSES)
else:
    model = resnet_v1(input_shape=input_shape, depth=DEPTH, num_classes=NUM_CLASSES)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

In [ ]:
import time

start_time = time.time()

# Prepare callbacks for model saving and for learning rate adjustment.
lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [lr_reducer, lr_scheduler]

# Run training, with or without data augmentation.
if not USE_AUGMENTATION:
    print('Not using data augmentation.')
    model.fit(xTrain, yTrain,
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              validation_data=(xVal, yVal),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)


    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(xTrain)
    
    with tensorflow.device('/gpu:0'):
        # Fit the model on the batches generated by datagen.flow().
        model.fit_generator(datagen.flow(xTrain, yTrain, batch_size=BATCH_SIZE),
                            validation_data=(xVal, yVal),
                            epochs=EPOCHS, verbose=1, workers=1,
                            callbacks=callbacks, use_multiprocessing=False)
    
elapsed_time = time.time() - start_time

In [ ]:
print("Elapsed time: {}".format(hms_string(elapsed_time)))
scores = model.evaluate(xTest, yTest, verbose=0)
print(scores)

In [ ]:
scores = model.evaluate(xTest, yTest, verbose=0)
print(scores)

model.save('my_model.h5')

### resnet2

In [22]:
# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
VERSION = 2

In [23]:
BATCH_SIZE = 20 #32  # orig paper trained all networks with batch_size=128
EPOCHS = 50 # 200
USE_AUGMENTATION = True

# Computed depth from supplied model parameter n
if VERSION == 1:
    DEPTH = COLORS * 6 + 2
elif VERSION == 2:
    DEPTH = COLORS * 9 + 2

In [24]:
# Create the neural network
if VERSION == 2:
    model_2 = resnet_v2(input_shape=input_shape, depth=DEPTH, num_classes=NUM_CLASSES)
else:
    model_2 = resnet_v1(input_shape=input_shape, depth=DEPTH, num_classes=NUM_CLASSES)

model_2.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model_2.summary()

Learning rate:  0.001
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 16) 0           batch_normalization[0][0]        
________________________________________________________________________

In [25]:
import time

start_time = time.time()

# Prepare callbacks for model saving and for learning rate adjustment.
lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [lr_reducer, lr_scheduler]

# Run training, with or without data augmentation.
if not USE_AUGMENTATION:
    print('Not using data augmentation.')
    model_2.fit(xTrain, yTrain,
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              validation_data=(xVal, yVal),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen2 = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)


    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen2.fit(xTrain)
    
    with tensorflow.device('/gpu:0'):
        # Fit the model on the batches generated by datagen.flow().
        model_2.fit_generator(datagen2.flow(xTrain, yTrain, batch_size=BATCH_SIZE),
                            validation_data=(xVal, yVal),
                            epochs=EPOCHS, verbose=1, workers=1,
                            callbacks=callbacks, use_multiprocessing=False)
    
elapsed_time = time.time() - start_time

Using real-time data augmentation.
Learning rate:  0.001
Epoch 1/50
 81/280 [=======>......................] - ETA: 3:17 - loss: 2.5518 - accuracy: 0.2549

KeyboardInterrupt: 

In [ ]:
print("Elapsed time: {}".format(hms_string(elapsed_time)))
scores = model_2.evaluate(xTest, yTest, verbose=0)
print(scores)
model_2.save("./model/seti-resnet_v2.h5")